# モデルの不公平性を検出して軽減する

機械学習モデルには意図しないバイアスが組み込まれていることがあり、*公平性* の問題につながる可能性があります。たとえば、糖尿病の可能性を予測するモデルは、一部の年齢層ではうまく機能しても、他の年齢層ではうまく機能しないかもしれません。そのため、一部の患者が不必要な検査を受けたり、糖尿病の診断を確定する検査を受けられなかったりする可能性があります。

このノートブックでは、**Fairlearn** パッケージを使用してモデルを分析し、年齢に基づいて患者の異なるサブセットの予測パフォーマンスの格差を探ります。

> **注**: Fairlearn パッケージとの統合は、現在プレビュー中です。予期しないエラーが発生する場合があります。

## 重要 - 公平性への配慮

> このノートブックは、Fairlearn パッケージと Azure Machine Learning との統合を探索するための実践的な演習として設計されています。しかし、組織やデータ サイエンス チームがツールを使用する前に、公平性に関して議論しなければならない考慮事項は非常に多くあります。公平性は、単にモデルを分析するツールを実行するだけにとどまらない、複雑な *社会技術的* 課題です。
>
> Microsoft Research は 1 行のコードが記述される前に行われる必要がある重要な議論の出発点となる [公平性チェックリスト](https://www.microsoft.com/en-us/research/publication/co-designing-checklists-to-understand-organizational-challenges-and-opportunities-around-fairness-in-ai/) を共同開発しました。

## 必要な SDK をインストールする

Azure Machine Learning で Fairlearn パッケージを使用するには、Azure Machine Learning と Fairlearn Python パッケージが必要なので、次のセルを実行して **azureml-contrib-fairness** パッケージがインストールされていることを確認します。 

In [ ]:
!pip show azureml-contrib-fairness

**fairlearn** パッケージ自体も必要です。インストールするには次のセルを実行します。

In [ ]:
!pip install --upgrade fairlearn==0.5.0

## モデルをトレーニングする

まず、糖尿病の可能性を予測するための分類モデルをトレーニングします。データを特徴およびラベルのトレーニング セットとテスト セットに分割するだけでなく、公平性を比較するデータの部分母集団を定義するために使用される *センシティブ* 特徴を抽出します。この例では、**Age** 列を使用して、50歳を超える患者と50歳以下の患者という二つのカテゴリを定義します。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# 糖尿病データセットを読み込む
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# 特徴とラベルを分離する
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
X, y = data[features].values, data['Diabetic'].values

# センシティブ特徴を入手する
S = data[['Age']].astype(int)
# 年齢層を表すために値を変更する
S['Age'] = np.where(S.Age > 50, 'Over 50', '50 or younger')

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.20, random_state=0, stratify=y)

# 分類モデルをトレーニングする
print("Training model...")
diabetes_model = DecisionTreeClassifier().fit(X_train, y_train)

print("Model trained.")

モデルのトレーニングが完了したので、Fairlearn パッケージを使用して、さまざまなセンシティブ特徴値に対する動作を比較できます。この例では、以下のことを行います。

- fairlearn **selection_rate** 関数を使用して、母集団全体の選択率 (正の予測の割合) を返します。
- **scikit-learn** メトリック関数を使用して、全体的な精度、再現率、適合率のメトリックを計算します。
- **MetricFrame** を使用して、**Age** のセンシティブ特徴の各年齢層の選択率、精度、再現率および適合率を計算します。パフォーマンス値の計算には、**fairlearn** と **scikit-learn** のメトリック関数を組み合わせて使用することに注意してください。

In [ ]:
from fairlearn.metrics import selection_rate, MetricFrame
from sklearn.metrics import accuracy_score, recall_score, precision_score

# 伏せられたテスト データの予測を取得する
y_hat = diabetes_model.predict(X_test)

# 全体的なメトリックを取得する
print("Overall Metrics:")
# Fairlearn から選択率を取得する
overall_selection_rate = selection_rate(y_test, y_hat) # Get selection rate from fairlearn
print("\tSelection Rate:", overall_selection_rate)
# scikit-learn から標準的なメトリックを取得する
overall_accuracy = accuracy_score(y_test, y_hat)
print("\tAccuracy:", overall_accuracy)
overall_recall = recall_score(y_test, y_hat)
print("\tRecall:", overall_recall)
overall_precision = precision_score(y_test, y_hat)
print("\tPrecision:", overall_precision)

# Fairlearn からセンシティブ グループのメトリックを取得する
print('\nMetrics by Group:')
metrics = {'selection_rate': selection_rate,
           'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score}

group_metrics = MetricFrame(metrics,
                             y_test, y_hat,
                             sensitive_features=S_test['Age'])

print(group_metrics.by_group)

これらのメトリックから、高齢患者の大部分が糖尿病であると予測されることがわかるはずです。*精度* は 2 つのグループでほぼ同じですが、*精度* と *再現率* を詳しく調べると、各年齢層でのモデルの予測精度に多少の格差があることがわかります。

このシナリオでは、*再現率* を検討します。このメトリックは、モデルによって正しく識別された陽性症例の割合を示します。つまり、実際に糖尿病にかかっているすべての患者のうち、モデルに検出された人数です。このモデルは、若年患者よりも高齢患者の方がこの点で優れた結果を示しました。

多くの場合、メトリックを視覚的に比較する方が簡単です。これを行うには、Fairlearn ダッシュボードを使用します。

1.次のセルを実行します (*将来の変更に関する警告が表示される可能性があることに注意してください。現時点では無視します*)。
2.ウィジェットが表示されたら、**はじめに** リンクを使用して視覚化の設定を開始します。
3.比較するセンシティブ特徴を選択します (この例では、次の 1 つだけです: **年齢**)。
4.比較するモデル パフォーマンス メトリックを選択します (この場合は二項分類モデルであるため、オプションは *精度*、*均衡の取れた精度*、*適合率*、*再現率* です)。**再現率** から始めます。
5.ダッシュボードの視覚化を表示します。次の内容が表示されます。
    - **パフォーマンスの格差** - *予測不足* (偽陰性) や *予測過剰* (偽陽性) などの部分母集団と比較した、選択したパフォーマンス メトリック。
    - **予測の格差** - 部分母集団あたりの陽性症例数の比較。
6.構成を編集して、異なるパフォーマンス メトリックに基づく予測を比較します。

In [ ]:
from fairlearn.widget import FairlearnDashboard

# Fairlearn の公平性ダッシュボードでこのモデルを表示し、表示される差異を確認する
FairlearnDashboard(sensitive_features=S_test, 
                   sensitive_feature_names=['Age'],
                   y_true=y_test,
                   y_pred={"diabetes_model": diabetes_model.predict(X_test)})

その結果、50 歳以上の患者の方が若年患者よりも選択率がはるかに高くなっています。しかし、実際には年齢は糖尿病の正真正銘の要因なので、高齢患者の方が陽性症例が多くなることが予想されます。

モデルのパフォーマンスを *精度* (つまり、モデルが正しく予測される割合) に基づいて判断すると、どちらの部分母集団に対してもほぼ同等に機能するように思われます。しかし、*適合率* と *再現率* のメトリックに基づくと、このモデルは 50 歳以上の患者でより良いパフォーマンスを示す傾向があります。

モデルをトレーニングするときに **年齢** 特徴を除外するとどうなるか見てみましょう。

In [ ]:
# 特徴とラベルを分離する
ageless = features.copy()
ageless.remove('Age')
X2, y2 = data[ageless].values, data['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train2, X_test2, y_train2, y_test2, S_train2, S_test2 = train_test_split(X2, y2, S, test_size=0.20, random_state=0, stratify=y2)

# 分類モデルをトレーニングする
print("Training model...")
ageless_model = DecisionTreeClassifier().fit(X_train2, y_train2)
print("Model trained.")

# Fairlearn の公平性ダッシュボードでこのモデルを表示し、表示される差異を確認する
FairlearnDashboard(sensitive_features=S_test2, 
                   sensitive_feature_names=['Age'],
                   y_true=y_test2,
                   y_pred={"ageless_diabetes_model": ageless_model.predict(X_test2)})

ダッシュボードのモデルを探索します。

*再現率* を見直す際には、このモデルが高齢患者の陽性症例を有意に過小評価しているため、格差は減少していますが、全体的な再現率も減少していることに注意します。**年齢** はトレーニングで使用された特徴ではありませんでしたが、このモデルは高齢患者と若年患者を予測する際に若干の格差を示しています。

このシナリオでは、**年齢** を削除するだけで、*再現率* の格差はわずかに減少しますが、*適合率* と *精度* の格差が大きくなります。これは、機械学習モデルに公平性を適用する際の重要な問題の 1 つを強調しています。特定のコンテキストにおいて *公平性* が何を意味するのかを明確にし、そのために最適化する必要があります。

## モデルを登録し、ダッシュボード データをワークスペースにアップロードします。

このノートブックでモデルをトレーニングし、ダッシュボードをローカルで確認しました。しかし、Azure Machine Learning ワークスペースにモデルを登録し、ダッシュボード データを記録する実験を作成すると、公平性分析を追跡して共有できるので便利かもしれません。

まずは元のモデル (特徴として **年齢** が含まれていました) を登録してみましょう。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
from azureml.core import Workspace, Experiment, Model
import joblib
import os

# 保存された構成ファイルから Azure ML ワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to work with', ws.name)

# トレーニング済みモデルを保存する
model_file = 'diabetes_model.pkl'
joblib.dump(value=diabetes_model, filename=model_file)

# モデルを登録する
print('Registering model...')
registered_model = Model.register(model_path=model_file,
                                  model_name='diabetes_classifier',
                                  workspace=ws)
model_id= registered_model.id


print('Model registered.', model_id)

これで、FairLearn パッケージを使用して 1 つ以上のモデルの二項分類グループ メトリック セットを作成し、Azure Machine Learning 実験を使用してメトリックをアップロードできるようになりました。

> **注**: これには時間がかかる場合があります。実験が完了すると、ダッシュボード データがダウンロードされて表示され、正常にアップロードされたことを確認できます。

In [ ]:
from fairlearn.metrics._group_metric_set import _create_group_metric_set
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

#  公平性を評価するモデルの辞書を作成する 
sf = { 'Age': S_test.Age}
ys_pred = { model_id:diabetes_model.predict(X_test) }
dash_dict = _create_group_metric_set(y_true=y_test,
                                    predictions=ys_pred,
                                    sensitive_features=sf,
                                    prediction_type='binary_classification')

exp = Experiment(ws, 'mslearn-diabetes-fairness')
print(exp)

run = exp.start_logging()

# Azure Machine Learning にダッシュボードをアップロードする
try:
    dashboard_title = "Fairness insights of Diabetes Classifier"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    # To test the dashboard, you can download it
    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
    print(downloaded_dict)
finally:
    run.complete()

上記のコードは、正常に完了したことを確認するためだけに、実験で生成されたメトリックをダウンロードしました。メトリックを実験にアップロードすることの本当の利点は、Azure Machine Learning Studio で FairLearn ダッシュボードを表示できるようになったことです。

下のセルを実行して実験の詳細を確認し、ウィジェットの **View Run details** (実行の詳細を表示する) リンクをクリックして Azure Machine Learning Studio での実行を確認します。次に、実験実行の **公平性** タブを表示してダッシュボードを表示します。ダッシュボードは、このノートブックで以前表示したウィジェットと同じように動作します。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Azure Machine Learning Studio の **モデル** ページでモデルを選択し、**公平性** タブを見ることで、公平性ダッシュボードを見つけることもできます。これにより、組織はトレーニングおよび登録するモデルの公平性分析のログを保持できます。

## モデルの不公正性を軽減する

これでモデルの公平性を分析できたので、FairLearn パッケージでサポートされている *軽減* 技術のいずれかを使用して、予測パフォーマンスと公平性のバランスを取るモデルを見つけることができます。

この演習では、**GridSearch** 機能を使用します。この機能は、データセット内のセンシティブ特徴 (この場合、年齢層) の予測パフォーマンスの格差を最小限に抑えるために、複数のモデルをトレーニングします。**EqualizedOdds** パリティ制約を適用してモデルを最適化します。これは、センシティブ特徴グループごとに、モデルが同じような真陽性率と偽陽性率を示すようにするものです。 

> *実行に時間がかかる場合があります*

In [ ]:
from fairlearn.reductions import GridSearch, EqualizedOdds
import joblib
import os

print('Finding mitigated models...')

# 複数のモデルをトレーニングする
sweep = GridSearch(DecisionTreeClassifier(),
                   constraints=EqualizedOdds(),
                   grid_size=20)

sweep.fit(X_train, y_train, sensitive_features=S_train.Age)
models = sweep.predictors_

# モデルを保存し、そこから予測を取得する(加えて、比較用に元の純粋なもの)
model_dir = 'mitigated_models'
os.makedirs(model_dir, exist_ok=True)
model_name = 'diabetes_unmitigated'
print(model_name)
joblib.dump(value=diabetes_model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
predictions = {model_name: diabetes_model.predict(X_test)}
i = 0
for model in models:
    i += 1
    model_name = 'diabetes_mitigated_{0}'.format(i)
    print(model_name)
    joblib.dump(value=model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
    predictions[model_name] = model.predict(X_test)


これで FairLearn ダッシュボードを使用して、軽減されたモデルを比較できるようになりました。

次のセルを実行し、ウィザードを使用して **再現率**.別の **年齢** を視覚化します。

In [ ]:
FairlearnDashboard(sensitive_features=S_test, 
                   sensitive_feature_names=['Age'],
                   y_true=y_test,
                   y_pred=predictions)

モデルは散布図に示されています。モデルを比較するには、予測の格差 (つまり選択率) または選択したパフォーマンス メトリックの格差 (この場合、*再現率*) を測定します。このシナリオでは、選択率の格差が予想されます (糖尿病では年齢*が*要因であることがわかっているので、年齢層が高いほど陽性症例が多くなります)。ここで注目したいのは、予測パフォーマンスの格差です。そこで、**再現率の不均衡** を測定するオプションを選択します。

グラフは、X 軸に全体的な *再現率* メトリック、Y 軸に再現率の格差を持つモデルのクラスターを示しています。したがって、理想的なモデル (再現率が高く、格差が小さい) は、プロットの右下にあります。特定のニーズに適した予測パフォーマンスと公平性のバランスを選択し、適切なモデルを選択してその詳細を確認できます。

強調すべき重要な点は、モデルに公平性の軽減を適用することは、全体的な予測パフォーマンスとセンシティブ特徴グループ間の格差との間のトレードオフであるということです。一般的には、モデルが母集団のすべてのセグメントに対して公平に予測することを保証するために、全体的な予測パフォーマンスを犠牲にする必要があります。

> **注**: *適合率* メトリックを表示すると、予測されるサンプルがないために適合率が 0.0 に設定されているという警告が表示される場合があります。これは無視してかまいません。

## Azure Machine Learning に軽減ダッシュボード メトリックをアップロードする

前述のように、軽減の実験を追跡することもできます。これを行うには、次を実行します。

1. GridSearch プロセスで検出されたモデルを登録します。
2. モデルのパフォーマンスおよび格差メトリックを計算します。
3. Azure Machine Learning の実験にメトリックをアップロードします。

In [ ]:
# モデルを登録する
registered_model_predictions = dict()
for model_name, prediction_data in predictions.items():
    model_file = os.path.join(model_dir, model_name + ".pkl")
    registered_model = Model.register(model_path=model_file,
                                      model_name=model_name,
                                      workspace=ws)
    registered_model_predictions[registered_model.id] = prediction_data

#  すべてのモデルの年齢特徴に基づいて、二項分類用のグループ メトリック セットを作成する
sf = { 'Age': S_test.Age}
dash_dict = _create_group_metric_set(y_true=y_test,
                                     predictions=registered_model_predictions,
                                     sensitive_features=sf,
                                     prediction_type='binary_classification')

exp = Experiment(ws, "mslearn-diabetes-fairness")
print(exp)

run = exp.start_logging()
RunDetails(run).show()

# Azure Machine Learning にダッシュボードをアップロードする
try:
    dashboard_title = "Fairness Comparison of Diabetes Models"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))
finally:
    run.complete()

> **注**: 予測されるサンプルがないために適合率が 0.0 に設定されているという警告が表示される場合があります。これは無視してかまいません。


実験が終了したら、ウィジェットの **View Run details** (実行の詳細を表示する) リンクをクリックして Azure Machine Learning Studio での実行を確認し、**公平性** タブの FairLearn ダッシュボードを表示します。